# 🎙️ Speech-to-Text (STT) 강의 노트

> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY`를 다시 설정해야 합니다.


## 1) 환경 준비 및 API 키 설정

In [ ]:
%pip install openai

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI
from pathlib import Path

client = OpenAI()
print("✅ 클라이언트 생성 완료")


## 2) STT 개요 및 지원 형식

**Speech-to-Text(STT)**는 사람의 말소리를 텍스트로 변환하는 기술입니다.

**사용 예시**
- 음성 기반 검색 시스템
- 실시간 자막 생성
- 음성 메모 변환

**OpenAI Audio API는 두 가지 speech-to-text 엔드포인트를 제공합니다:**
- **`transcriptions`**: 오디오를 해당 언어의 텍스트로 변환
- **`translations`**: 오디오를 영어 텍스트로 번역 및 변환

**지원 모델**
- **`whisper-1`**: 오픈소스 Whisper 모델 기반 (기본 모델)
- **`gpt-4o-transcribe`**: 고품질 전사 모델
- **`gpt-4o-mini-transcribe`**: 경량 고품질 전사 모델
- **`gpt-4o-transcribe-diarize`**: 발화자 구분 기능 포함 모델

**파일 제한사항**
- **파일 크기**: 최대 25MB
- **지원 형식**: `mp3`, `mp4`, `mpeg`, `mpga`, `m4a`, `wav`, `webm`

**지원 언어**
- **한국어**, 영어 등 98개 언어

## 3) Transcriptions 엔드포인트 기본 사용법

### 기본 흐름
- `client.audio.transcriptions.create`로 오디오 파일과 모델을 지정해 텍스트를 얻습니다.
- 실습 모델: `gpt-4o-mini-transcribe`
- 응답은 기본적으로 JSON(`response.text`)이며, `response_format="text"`로 순수 텍스트를 받을 수 있습니다.

### 주요 파라미터
- **`file`**: 변환할 오디오 파일 객체 (파일 이름이 아님)
- **`model`**: 사용할 모델 (`gpt-4o-transcribe`, `gpt-4o-mini-transcribe`, `whisper-1` 등)
- **`language`**: 입력 오디오의 언어 (ISO-639-1 형식은 정밀도와 지연을 개선합니다, 예: `"ko"`, `"en"`)
    - [ISO-639-1 형식](https://en.wikipedia.org/wiki/List_of_ISO_639_language_codes)
- **`prompt`**: (선택) 모델의 스타일을 안내하거나 이전 오디오 세그먼트를 계속하기 위한 텍스트, audio language와 매치해야함
- **`response_format`**: 출력 형식 (`json`, `text`, `srt`, `verbose_json`, `vtt`, `diarized_json`)
- **`temperature`**: 0~1 사이의 샘플링 temperature (기본값: 0)


In [ ]:
def transcribe_basic(audio_path):
    """기본 전사 함수 - gpt-4o-transcribe 모델 사용"""
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="gpt-4o-transcribe",
            file=audio_file,
        )
    
    print("전사 결과 예시:\n", transcription.text[:200])
    return transcription.text

# 사용 예시:
# transcribe_basic("data/sample_lecture.mp3")


## 4) 고급 옵션: 응답 포맷 · 프롬프트 · 로그확률

### 응답 포맷 (response_format)
- **`json`**: 기본 JSON 형식 (텍스트 포함)
- **`text`**: 순수 텍스트만 반환
- **`verbose_json`**: 타임스탬프 등 추가 메타데이터 포함
- **`diarized_json`**: 발화자 구분 정보 포함 (gpt-4o-transcribe-diarize 전용)
- **`srt`**: 자막 파일 형식
- **`vtt`**: WebVTT 자막 형식

### 프롬프트 (prompt)
- 모델이 잘못 인식하는 용어/고유명사를 안내해 정확도 향상
- 이전 오디오 세그먼트의 텍스트를 전달해 맥락 유지
- `whisper-1`은 처음 224 토큰만 고려
- `gpt-4o-transcribe`는 더 긴 프롬프트 지원

### 로그확률 (logprobs)
- `include=["logprobs"]`: 토큰별 확률을 확인해 신뢰도 분석
- `gpt-4o-transcribe` 및 `gpt-4o-mini-transcribe`에서 지원


In [ ]:
def transcribe_with_options(audio_path, *, prompt=None, response_format="json", language=None):
    """고급 옵션을 사용한 전사 함수"""
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")

    kwargs = {
        "model": "gpt-4o-transcribe",
        "response_format": response_format,
    }
    
    if prompt:
        kwargs["prompt"] = prompt
    if language:
        kwargs["language"] = language  # ISO-639-1 형식 (예: "ko", "en")
    if response_format == "json":
        kwargs["include"] = ["logprobs"]

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            **kwargs,
        )

    print("텍스트 예시:", transcription.text[:120])
    if response_format == "json" and getattr(transcription, "logprobs", None):
        print("첫 토큰 로그확률:", transcription.logprobs[0])
    
    return transcription

# 사용 예시:
# transcribe_with_options(
#     "data/sample_lecture.mp3", 
#     prompt="This is a talk about GPT-4o and Sora.",
#     language="en"
# )


## 5) 발화자 구분(Diarization)

### 개요
`gpt-4o-transcribe-diarize` 모델은 회의나 대화에서 각 발화자를 구분하여 전사할 수 있습니다.

### 주요 특징
- `response_format="diarized_json"`을 통해 `speaker`, `start`, `end` 정보를 포함한 세그먼트 제공
- 30초 이상 파일은 `chunking_strategy="auto"` 지정 필수 (권장)
- `known_speaker_names[]`와 `known_speaker_references[]`로 사전 등록한 음성과 발화자 매칭 가능
- 스트리밍(`stream=True`) 시 세그먼트 완료 시점마다 `transcript.text.segment` 이벤트 발생

### 제한사항
- `prompt`, `logprobs`, `timestamp_granularities[]` 파라미터 미지원
- Realtime API에서는 아직 지원되지 않음


In [ ]:
def diarize_demo(audio_path, known_speakers=None):
    """발화자 구분 전사 함수"""
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")

    kwargs = {
        "model": "gpt-4o-transcribe-diarize",
        "response_format": "diarized_json",
        "chunking_strategy": "auto",
    }
    
    # 알려진 발화자 정보가 있으면 추가
    if known_speakers:
        import base64
        known_speaker_names = []
        known_speaker_references = []
        
        for name, ref_path in known_speakers.items():
            known_speaker_names.append(name)
            with open(ref_path, "rb") as f:
                ref_data = base64.b64encode(f.read()).decode("utf-8")
                known_speaker_references.append(f"data:audio/wav;base64,{ref_data}")
        
        kwargs["extra_body"] = {
            "known_speaker_names": known_speaker_names,
            "known_speaker_references": known_speaker_references,
        }

    with path.open("rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            file=audio_file,
            **kwargs,
        )

    print("발화자별 전사 결과 (처음 5개 세그먼트):")
    for segment in transcript.segments[:5]:
        print(f"{segment.speaker}: {segment.text} ({segment.start:.1f}s ~ {segment.end:.1f}s)")
    
    return transcript

# 사용 예시:
# diarize_demo("data/meeting.wav")
# 
# 알려진 발화자가 있는 경우:
# diarize_demo("data/meeting.wav", known_speakers={
#     "agent": "data/agent.wav",
#     "customer": "data/customer.wav"
# })


## 6) Translations 엔드포인트

### 개요
`translations` 엔드포인트는 비영어 음성을 영어 텍스트로 번역합니다.

### 특징
- **`whisper-1` 모델 전용**: 현재 `whisper-1`만 지원
- **출력 언어**: 원문 언어가 무엇이든 출력은 항상 영어
- **입력 언어**: 지원되는 모든 언어의 오디오를 영어로 번역 가능

### 사용 사례
- 다국어 회의록을 영어로 통일
- 글로벌 콘텐츠 자막 생성
- 언어 장벽 없는 음성 번역 서비스


In [ ]:
def translate_demo(audio_path):
    """비영어 오디오를 영어로 번역하는 함수"""
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {path}")

    with path.open("rb") as audio_file:
        translation = client.audio.translations.create(
            model="whisper-1",
            file=audio_file,
        )

    print("영어 번역 결과:", translation.text[:200])
    return translation.text

# 사용 예시:
# translate_demo("data/german.mp3")
# translate_demo("data/korean_speech.wav")


## 10) 품질 향상을 위한 프롬프트/후처리 전략

### 프롬프트 활용 방법

#### 1. 고유명사/약어 보정
`prompt` 파라미터에 올바른 철자 리스트를 제공하여 인식 정확도 향상

```python
prompt = "DALL·E, GPT-3, ChatGPT, OpenAI"
```

#### 2. 이전 세그먼트 맥락 유지
긴 오디오를 분할할 때 이전 세그먼트의 텍스트를 prompt로 전달

#### 3. 구두점/스타일 안내
- 구두점 포함: `prompt="Hello, welcome to my lecture."`
- 필러 단어 유지: `prompt="Umm, let me think like, hmm..."`
- 언어 스타일 지정: 간체/번체 중국어 등

#### 4. 제한사항
- `whisper-1`: 처음 224 토큰만 고려 (다국어는 커스텀 토크나이저 사용)
- `gpt-4o-transcribe`: 더 긴 프롬프트 지원, 더 나은 지시 따르기

### GPT-4 기반 후처리

전사 결과를 GPT-4에 전달하여 품질 향상:

1. **오타 수정**: 잘못 인식된 단어 보정
2. **맞춤법 보정**: 문법 및 철자 교정
3. **스타일 통일**: 일관된 형식으로 변환
4. **확장성**: 긴 컨텍스트 처리 가능

```python
system_prompt = """
You are a helpful assistant. Your task is to correct any spelling 
discrepancies in the transcribed text. Make sure that the names of 
the following products are spelled correctly: DALL·E, GPT-3, ChatGPT.
Only add necessary punctuation such as periods, commas, and capitalization.
"""

transcript = transcribe_audio(audio_file)
corrected = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": transcript}
    ]
)
```

> **실습 연결**: 학생들에게 prompt 튜닝과 후처리 파이프라인을 직접 실험하게 하고, 품질 향상 전/후 비교를 과제로 제시해 보세요.


## 11) 실전 활용 예시

### 회의록 자동 작성
```python
# 발화자 구분을 활용한 회의록 생성
transcript = diarize_demo("data/meeting.wav")
for segment in transcript.segments:
    print(f"[{segment.speaker}] {segment.text}")
```

### 다국어 콘텐츠 자막 생성
```python
# 한국어 오디오를 영어 자막으로 변환
english_subtitle = translate_demo("data/korean_video.wav")
```

### 영상 편집 자동화
```python
# 단어 단위 타임스탬프로 정확한 편집 지점 찾기
transcription = transcribe_with_timestamps("data/video_audio.wav")
# 특정 단어의 위치를 찾아 해당 프레임 편집
```

### 실시간 자막 서비스
- 스트리밍 API를 활용한 실시간 자막 생성
- Realtime API로 WebSocket 기반 실시간 전사


In [ ]:
## 12) 샘플 데이터셋

### 채용면접 인터뷰 데이터
- **링크**: [AI Hub - 채용면접 데이터셋](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71592)
- **용도**: 발화자 구분(Diarization) 실습에 적합
- **형식**: 면접관과 지원자의 대화 녹음

### 오디오 파일 분할 예시 코드
아래 코드는 오디오 파일을 정해진 시간에 따라 분절하여 별도의 파일로 저장합니다:

```python
from pydub import AudioSegment

# 오디오 파일 로드
audio = AudioSegment.from_wav("interview.wav")

# 5분(300초) 단위로 분할
segment_length = 5 * 60 * 1000  # 밀리초

for i, start_time in enumerate(range(0, len(audio), segment_length)):
    segment = audio[start_time:start_time + segment_length]
    segment.export(f"interview_part_{i+1}.wav", format="wav")
```


> 💡 **강의 마무리**
> - 프롬프트 튜닝과 후처리 파이프라인을 통해 실제 프로젝트에서 활용할 수 있는 품질 향상 기법을 강조하세요.


### 참고 자료
- https://platform.openai.com/docs/guides/speech-to-text
- https://platform.openai.com/docs/api-reference/audio